In [ ]:
import requests
from requests.models import PreparedRequest
import os
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()

### Important URL's

In [ ]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

# Refresh Token

Step for obtaining a refresh token that allows to "read_all" instead of just reading a instance at a time (default). Requires a Auth on a webbrowser, which results in the code variable.

In [ ]:
"""
https://www.strava.com/oauth/authorize?client_id=115052&redirect_uri=http://localhost&response_type=code&approval_prompt=auto&scope=activity:read_all
"""

In [ ]:
code = "0e3c260a8910acf7387befd3d38d4db2c4a009be"

In [ ]:
payload = {
    'client_id': '115052',
    'client_secret': str(os.getenv("client_secret")),
    'code': code,
    'grant_type': "authorization_code"
}

pos = requests.post(auth_url,data=payload).json()

In [ ]:
refresh_token = pos['refresh_token']
access_token = pos['access_token']

# New Access Token

In [ ]:
payload= {
    'client_id': '115052',
    'client_secret': str(os.getenv("client_secret")),
    'refresh_token': refresh_token, #Defend on the data permissions
    'grant_type': "refresh_token",
    'f': 'json' 
}


In [ ]:
print("Requesting Token... \n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

print("Access Token = {}\n".format(access_token))
header = {'Authorization': 'Bearer ' + str(access_token)}

In [ ]:
print("Requesting pages (200 activites per full page)... ")

activities_df = []

page = 1
page_non_empty = True

while page_non_empty:  
    param = {'per_page': 200, 'page': page}
    
    activities_json = requests.get(activites_url, headers=header, params=param).json()
    print(activities_json)
    if not activities_json:
        page_non_empty=False
        break
    activities_df.append(pd.json_normalize(activities_json))
    
    page+=1
print("\n", len(activities_df), "activties downloaded!")

# Obtain the "parts"

In [ ]:
df = pd.concat([activities_df[0],activities_df[1]],axis=0, ignore_index=True)

In [ ]:
df.iloc[1:2]

In [ ]:
(df['route_id'])

In [ ]:
df['deviation_heartrate'] = None
df['deviation_cadence'] = None
#df['deviation_speed'] = None


In [ ]:
import numpy as np
import time

# Define your rate limit values
request_limit = 190
time_limit = 900  # 15 minutes in seconds

# Initialize variables to keep track of requests and timing
requests_made = 0
start_time = time.time()

retry_interval = 60*5  # 60 seconds

for index,row in df.iterrows():
    #index=9

    id = row['id']
    print("Current Activity ID, ", id)
    url = f"https://www.strava.com/api/v3/activities/{id}/streams"

    #latlong = requests.get(url, headers=header, params={'keys':['latlng']}).json()[0]['data']
    #time_list = requests.get(url, headers=header, params={'keys':['time']}).json()[1]['data']
    #altitude = requests.get(url, headers=header, params={'keys':['altitude']}).json()[1]['data']
    #hr = requests.get(url, headers=header, params={'keys':['heartrate']}).json()[1]['data']



    # Devia implementar algo para quase falhasse nao re-começar
    # Initialize a flag to track whether data retrieval was successful
    data_retrieved = False

    while not data_retrieved:
        try:
            hr = requests.get(url, headers=header, params={'keys': ['heartrate']}).json()[1]['data']
            requests_made += 1
            df.loc[df['id'] == id, 'deviation_heartrate'] = np.std(hr)
            data_retrieved = True  # Set the flag to True when data is successfully retrieved
        except (IndexError):
            # Handle the case where 'heartrate' data is not present or any other error occurs
            hr = None
            data_retrieved = True 
            print('Activity {}, does not have heart rate info.'.format(id)) 
            
            
        except KeyError:
            print('Retrying in {} seconds...'.format(retry_interval))
            time.sleep(retry_interval)


    if (row['type'] == 'Run'):
        data_retrieved = False  # Reset the flag for cadence data
        
        while not data_retrieved:
            try:
                cadence = requests.get(url, headers=header, params={'keys': ['cadence']}).json()[1]['data']
                requests_made += 1
                df.loc[df['id'] == id, 'deviation_cadence'] = np.std(cadence)
                data_retrieved = True  # Set the flag to True when data is successfully retrieved
            except (IndexError):
                # Handle the case where 'cadence' data is not present
                cadence = None
                data_retrieved = True 
                print('Activity {}, does not have cadence info'.format(id))
        
            except KeyError:
                print('Retrying in {} seconds...'.format(retry_interval))
                time.sleep(retry_interval)

"""
    try:
        hr = requests.get(url, headers=header, params={'keys': ['heartrate']}).json()[1]['data']
        requests_made += 1
        df.loc[df['id'] == id, 'deviation_heartrate'] = np.std(hr)
    except (IndexError, KeyError):
        # Handle the case where 'heartrate' data is not present or any other error occurs
        hr = None
        print('Activity {}, does not have heart rate info'.format(id))

    if (row['type'] == 'Run'):
        try:
            cadence = requests.get(url, headers=header, params={'keys':['cadence']}).json()[1]['data']
            requests_made += 1
            df.loc[df['id']== id ,'deviation_cadence'] = np.std(cadence)
        except IndexError:
        # Handle the case where 'cadence' data is not present
            cadence = None
            print('Activity {}, does not have cadence info'.format(id))
"""
        
    
"""
    if requests_made >= request_limit:
        # Calculate the time passed since the start
        elapsed_time = time.time() - start_time

        # If less than the time limit has passed, sleep for the remaining time
        if elapsed_time < time_limit:
            sleep_time = time_limit - elapsed_time
            print(f"Reached rate limit. Sleeping for {sleep_time} seconds.")
            time.sleep(sleep_time)

        # Reset request count and start time
        requests_made = 0
        start_time = time.time()
"""

"""
# Create dataframe to store data 'neatly'
data = pd.DataFrame([*latlong], columns=['lat','long'])
data['altitude'] = altitude
start = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%SZ")
data['time'] = [(start+timedelta(seconds=t)) for t in time_list]

gpx = gpxpy.gpx.GPX()
# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)
# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)
# Create points:
for idx in data.index:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
                data.loc[idx, 'lat'],
                data.loc[idx, 'long'],
                elevation=data.loc[idx, 'altitude'],
                time=data.loc[idx, 'time']
    ))
# Write data to gpx file
with open('Data/{}.gpx'.format(id), 'w') as f:
    f.write(gpx.to_xml())
"""

In [ ]:
import numpy as np
from scipy import stats as sc

print("Standart deviation: ", np.std(hr[1]['data']))
print("interquartile range: ", sc.stats.iqr(hr[1]['data']))



In [ ]:
df.to_csv("my_activities", sep=',', index=False, encoding='utf-8')

In [ ]:
path_gear = "/Users/diogogoncalves/Documents/GitProjects/Personal/athletic-analysis/test_tutorial/Data/my_gear"
gear_df = pd.read_csv(path_gear, sep=',')

In [ ]:
gear_df['Name'] = None
gear_url = "https://www.strava.com/api/v3/gear/{}"
for id in gear_df['gear_id'].unique():
   
    try:
        gear_info = requests.get(gear_url.format(id), headers=header).json()
        
    except:
        print("Something failed lol")


    gear_df.loc[gear_df['gear_id']== id, 'Name'] = str(gear_info['brand_name'] + " " + gear_info['model_name'])

In [ ]:
gear_df.to_csv("my_gear", sep=',', index=False, encoding='utf-8')